In [1]:
import json

def get_test_queries(line_nums):
    if isinstance(line_nums, int):
        line_nums = {line_nums}
    line_nums = set(line_nums)
    test_queries = []
    with open('test_qas.json') as f:
        line_num = 1
        for line in f:
            if line_num in line_nums:
                test_queries.append(json.loads(line))
                line_nums.remove(line_num)
            if len(line_nums) == 0:
                break
            line_num += 1
    return test_queries

# q = get_test_queries(1)[0]
# del q['origin_answers']
# q

In [2]:
import requests
ENTITY_RESOLVER_URL = "http://fuzzy-entity-resolver/resolve"

def resolve_entity(entity):
    entity_resp = requests.get(ENTITY_RESOLVER_URL, params={'query': entity}).json()
    tags = entity_resp['tags']


    # Filter out tags with low confidence
    tags = [t for t in tags if t['confidence'] >= 0.9]
    for t in tags:
        t['confidence'] = 1
    # print(tags)
    return tags

In [3]:
import requests

KBQA_URL = 'http://kbqa-service/HEALS/api/v1.0'#
# Query KBQA using the entities and expanded query
#

def run_query(q):
    print(f'asking query {q["qId"]}: {q["qOriginText"]}')
    kbqa_query_data = {
        "intent": "constraint",
        "output": "N/A",
        "question": q['qText'],
        # we don't need to expand the query again
        "skip_expansion": True,
        "lookup_recipes": True,
        "persona": q['persona'],
        "entities": [tag for e in q['entities'] for tag in resolve_entity(e[0])],
        "constrained_entities": q['persona']['constrained_entities'],
        #"similar_recipes": similar_recipes
    }
    kbqa_resp = requests.post(f"{KBQA_URL}/kbqa", json=kbqa_query_data)
    return kbqa_resp.json()


In [ ]:
import os
import re
# query_set = {4,5,7,9}
qline_regex = re.compile(r'^\s*(\d+)\s.*')
query_nums = []
for filename in os.listdir('./selected_questions'):
    with open(f'./selected_questions/{filename}') as file:
        for line in file:
            m = qline_regex.match(line)
            if m:
                query_nums.append(int(m[1]))
len(query_nums)

In [ ]:
queries = get_test_queries(query_nums)
for query in queries:
    query['kbqa_answer'] = run_query(query)
# queries[0]

asking query constraint-qas-test-00003: What venezuelan dishes can I make that do not contain chicken drumsticks?
asking query constraint-qas-test-00004: What low fat crawfish dishes can I make without alligator sausage?
asking query constraint-qas-test-00006: What are new zealand dishes which don't consist of chives?
asking query constraint-qas-test-00008: What are clams recipes which do not have ingredient baby clams?


In [5]:
results_markdown = "# test query results\n\n"

for query in queries:
    results_markdown += f"## {query['qId']}\n\n{query['qOriginText']}\n\n"
    
    for recipe in query['kbqa_answer']['answer_recipe_list']:
        results_markdown += f"1. **{recipe['name']}**\n"
        results_markdown += "| | |\n| --- | --- |\n"
        for name, value in recipe['nutrients'].items():
            results_markdown += f"| {name} | {value} |\n"
        results_markdown += '\n\tIngredients:\n'
        for ing in recipe['ingredients']:
            results_markdown += f'\t* {ing["name"]}\n'
    results_markdown += "\n"

from IPython.display import display, Markdown, Latex
display(Markdown(results_markdown))

# test query results

## constraint-qas-test-00003

What venezuelan dishes can I make that do not contain chicken drumsticks?

1. **Hallacas**
| | |
| --- | --- |
| calories | 2297.3 |
| carbohydrates | 118.7 |
| fat | 168.8 |
| protein | 0.0 |

	Ingredients:
	* beef
	* boiling water
	* green bell peppers
	* olive oil
	* onions
	* parsley
	* pork
	* salt
	* seedless raisins
	* stuffed olives
	* tomatoes
	* vinegar
	* butter
	* water
	* canned chick   peas
	* capers
	* cornmeal
	* dried chilies
	* eggs
	* garlic cloves
	* granulated sugar
1. **Venezuelan Shredded Beef**
| | |
| --- | --- |
| calories | 311.9 |
| carbohydrates | 9.8 |
| fat | 18.9 |
| protein | 0.0 |

	Ingredients:
	* beef broth
	* black pepper
	* flank steaks
	* garlic cloves
	* red bell pepper
	* salt
	* sweet paprika
	* tomatoes
	* vegetable oil
	* yellow onions

## constraint-qas-test-00004

What low fat crawfish dishes can I make without alligator sausage?

1. **Seafood Etouffee**
| | |
| --- | --- |
| calories | 883.2 |
| carbohydrates | 37.6 |
| fat | 58.4 |
| protein | 0.0 |

	Ingredients:
	* kitchen bouquet
	* butter
	* shrimp bouillon cube
	* vegetable bouillon cube
	* celery
	* celery soup
	* cooked rice
	* green onions
	* green pepper
	* mushroom soup
	* onions
	* shrimp
1. **Crayfish Etouffee**
| | |
| --- | --- |
| calories | 390.0 |
| carbohydrates | 13.3 |
| fat | 24.7 |
| protein | 0.0 |

	Ingredients:
	* kitchen bouquet
	* black pepper
	* scallion
	* sugar
	* tomato paste
	* water
	* white wine
	* butter
	* cayenne pepper
	* cornstarch
	* crawfish tail meat
	* garlic cloves
	* green peppers
	* onions
	* salt
1. **Crawfish Etouffee**
| | |
| --- | --- |
| calories | 821.0 |
| carbohydrates | 20.8 |
| fat | 57.2 |
| protein | 0.0 |

	Ingredients:
	* kitchen bouquet
	* bell pepper
	* cream of shrimp soup
	* garlic
	* garlic powder
	* onion
	* parsley flakes
	* salt
	* steak seasoning
	* black pepper
	* butter
	* cayenne pepper
	* celery
	* condensed golden mushroom soup
	* crab boil
	* crawfish tails
	* cream of celery soup

## constraint-qas-test-00006

What are new zealand dishes which don't consist of chives?

1. **Hot Prunes and Custard**
| | |
| --- | --- |
| calories | 580.5 |
| carbohydrates | 123.7 |
| fat | 7.7 |
| protein | 0.0 |

	Ingredients:
	* custard
	* prunes
	* water
1. **Peaches or Pears and Custard**
| | |
| --- | --- |
| calories | 556.5 |
| carbohydrates | 117.6 |
| fat | 7.6 |
| protein | 0.0 |

	Ingredients:
	* custard
	* peach halves in syrup
1. **Yogurt Berry Fool**
| | |
| --- | --- |
| calories | 171.5 |
| carbohydrates | 25.3 |
| fat | 5.8 |
| protein | 0.0 |

	Ingredients:
	* custard
	* frozen mixed berries
	* hazelnuts
	* meringue nests
	* vanilla yogurt

## constraint-qas-test-00008

What are clams recipes which do not have ingredient baby clams?

1. **New England Clambake in a Kettle**
| | |
| --- | --- |
| calories | 904.5 |
| carbohydrates | 68.6 |
| fat | 20.9 |
| protein | 0.0 |

	Ingredients:
	* cheesecloth
	* chorizo sausage
	* clams
	* corn on the cob
	* ground pork breakfast sausage
	* hot dog
	* live lobster
	* onion
	* white fish fillets
	* white potatoes
1. **White Clam Pizza**
| | |
| --- | --- |
| calories | 339.0 |
| carbohydrates | 17.9 |
| fat | 18.9 |
| protein | 0.0 |

	Ingredients:
	* clams
	* extra virgin olive oil
	* flour
	* fresh thyme leaves
	* garlic
	* parmesan cheese
	* pepper
	* pizza dough
	* salt
1. **Tuscan Red Clam Sauce**
| | |
| --- | --- |
| calories | 575.8 |
| carbohydrates | 10.9 |
| fat | 54.8 |
| protein | 0.0 |

	Ingredients:
	* clams
	* dry white wine
	* garlic cloves
	* ground black pepper
	* hot pepper
	* olive oil
	* onion
	* parsley
	* red pear tomatoes
	* salt
1. **Spaghetti Alle Vongole (Spaghetti With Clams)**
| | |
| --- | --- |
| calories | 875.8 |
| carbohydrates | 98.0 |
| fat | 31.4 |
| protein | 0.0 |

	Ingredients:
	* clams
	* garlic cloves
	* hot pepper
	* olive oil
	* spaghetti
	* tomatoes
1. **Gnocchi With Garlicky Red Clam Sauce**
| | |
| --- | --- |
| calories | 145.8 |
| carbohydrates | 16.3 |
| fat | 3.5 |
| protein | 0.0 |

	Ingredients:
	* clams
	* crushed tomatoes
	* fresh basil
	* garlic
	* gnocchi
	* olive oil
	* onions
1. **Thick \'n\' Quick Clam Chowder**
| | |
| --- | --- |
| calories | 345.1 |
| carbohydrates | 19.8 |
| fat | 23.1 |
| protein | 0.0 |

	Ingredients:
	* clams
	* condensed cheddar cheese soup
	* condensed cream of celery soup
	* cream of onion soup
	* half   and   half cream
1. **Corn, Clam, and Mussel Chowder**
| | |
| --- | --- |
| calories | 329.3 |
| carbohydrates | 39.3 |
| fat | 7.2 |
| protein | 0.0 |

	Ingredients:
	* minced clams
	* mussels
	* 2%25 low   fat milk
	* all   purpose flour
	* onions
	* red potatoes
	* salt
	* thyme
	* bacon
	* celery
	* clam juice
	* fresh corn kernels
	* fresh thyme sprigs
	* half   and   half
1. **Parongah**
| | |
| --- | --- |
| calories | 1067.0 |
| carbohydrates | 11.4 |
| fat | 67.9 |
| protein | 0.0 |

	Ingredients:
	* canned clams
	* cheddar cheese
	* eggs
	* olive oil
	* salt
	* sausages
1. **Clams in Olive Oil With Jamon and Pine Nuts**
| | |
| --- | --- |
| calories | 919.1 |
| carbohydrates | 9.4 |
| fat | 85.1 |
| protein | 0.0 |

	Ingredients:
	* clams
	* crusty bread
	* extra virgin olive oil
	* fresh flat   leaf parsley
	* garlic cloves
	* pine nuts
	* serrano ham
1. **Clam Stuffed Pasta Shells**
| | |
| --- | --- |
| calories | 284.5 |
| carbohydrates | 4.8 |
| fat | 22.4 |
| protein | 0.0 |

	Ingredients:
	* clams
	* cream cheese
	* garlic cloves
	* jumbo pasta shells
	* parsley
	* prepared horseradish
	* salt and pepper
	* vegetable oil
1. **Linda\'s New England Fried Clams**
| | |
| --- | --- |
| calories | 674.5 |
| carbohydrates | 47.8 |
| fat | 21.1 |
| protein | 0.0 |

	Ingredients:
	* clams
	* cornflour
	* eggs
	* evaporated milk
	* ketchup
	* lemon wedges
	* salt
	* tartar sauce
	* vegetable oil
1. **Clay Pot San Francisco Clambake**
| | |
| --- | --- |
| calories | 246.6 |
| carbohydrates | 15.3 |
| fat | 8.6 |
| protein | 0.0 |

	Ingredients:
	* clams
	* dried basil
	* parsley
	* scallops
	* shrimp
	* dry white wine
	* garlic cloves
	* green pepper
	* green tomatoes
	* lemon
	* olive oil
	* onion
	* oregano
1. **Simple Clam Sauce**
| | |
| --- | --- |
| calories | 971.3 |
| carbohydrates | 89.0 |
| fat | 56.7 |
| protein | 0.0 |

	Ingredients:
	* clams
	* fresh parsley
	* garlic cloves
	* linguine
	* olive oil
	* parmesan cheese
	* salt and pepper
1. **Lighten up Linguine With Clams**
| | |
| --- | --- |
| calories | 664.9 |
| carbohydrates | 104.6 |
| fat | 17.7 |
| protein | 0.0 |

	Ingredients:
	* chopped clams with juice
	* fresh mushrooms
	* fresh parsley
	* garlic
	* light margarine
	* linguine
	* olive oil
	* parmesan cheese
	* salt
	* white pepper
1. **Paccheri and Clams**
| | |
| --- | --- |
| calories | 672.5 |
| carbohydrates | 71.9 |
| fat | 23.2 |
| protein | 0.0 |

	Ingredients:
	* clams
	* crushed red pepper flakes
	* flat leaf parsley
	* garlic cloves
	* kosher salt
	* olive oil
	* pasta
	* white wine
1. **DeAnn\'s Clam Fritters**
| | |
| --- | --- |
| calories | 757.0 |
| carbohydrates | 93.3 |
| fat | 12.9 |
| protein | 0.0 |

	Ingredients:
	* clams
	* egg
	* flour
	* garlic cloves
	* milk
	* onions
1. **Stuffed Clams Roasted in the Coals**
| | |
| --- | --- |
| calories | 84.7 |
| carbohydrates | 11.6 |
| fat | 2.7 |
| protein | 0.0 |

	Ingredients:
	* cherrystone clams
	* lime
	* monterey jack cheese
	* oyster sauce
	* pico de gallo
	* white wine
1. **Grilled Clams With Garlicky White Wine Sauce**
| | |
| --- | --- |
| calories | 223.8 |
| carbohydrates | 6.9 |
| fat | 6.4 |
| protein | 0.0 |

	Ingredients:
	* clams
	* dry white wine
	* extra virgin olive oil
	* fresh parsley
	* garlic cloves
	* horseradish
	* lemon wedges
	* salt and pepper
	* unsalted butter
1. **Linguine with Clam Sauce**
| | |
| --- | --- |
| calories | 639.9 |
| carbohydrates | 88.2 |
| fat | 20.5 |
| protein | 0.0 |

	Ingredients:
	* clams
	* fresh parsley
	* garlic cloves
	* linguine
	* olive oil
	* red pepper flakes
	* water
1. **Zuppa Di Vongole (Clam Soup)**
| | |
| --- | --- |
| calories | 354.2 |
| carbohydrates | 33.4 |
| fat | 17.4 |
| protein | 0.0 |

	Ingredients:
	* clams
	* garlic clove
	* olive oil
	* parsley
	* salt and pepper
	* toast
	* tomatoes
	* water
1. **Quick Clam Chowder**
| | |
| --- | --- |
| calories | 155.6 |
| carbohydrates | 13.6 |
| fat | 6.7 |
| protein | 0.0 |

	Ingredients:
	* clams
	* condensed cream of celery soup
	* condensed new england clam chowder
	* fresh parsley
	* milk
	* water
1. **Clams in Tomato Sauce**
| | |
| --- | --- |
| calories | 800.9 |
| carbohydrates | 41.2 |
| fat | 19.0 |
| protein | 0.0 |

	Ingredients:
	* clams
	* fresh ground black pepper
	* garlic cloves
	* lemons
	* olive oil
	* pesto sauce
	* sea salt
	* shallots
	* sherry wine
	* tomatoes
1. **Portuguese Clams**
| | |
| --- | --- |
| calories | 300.3 |
| carbohydrates | 17.9 |
| fat | 9.2 |
| protein | 0.0 |

	Ingredients:
	* cherrystone clams
	* cilantro
	* diced tomatoes
	* garlic cloves
	* olive oil
	* onion
	* sea salt
	* white wine
1. **Mainiac Linguine With Red Clam Sauce**
| | |
| --- | --- |
| calories | 420.0 |
| carbohydrates | 60.7 |
| fat | 9.2 |
| protein | 0.0 |

	Ingredients:
	* clams
	* flat leaf parsley
	* fresh basil
	* garlic cloves
	* linguine
	* olive oil
	* pepper
	* salt
	* tomato sauce
	* tomatoes with juice
1. **Ziti Rigati With Flounder and Clams**
| | |
| --- | --- |
| calories | 153.7 |
| carbohydrates | 20.2 |
| fat | 1.5 |
| protein | 0.0 |

	Ingredients:
	* canned clams
	* cayenne pepper
	* water
	* ziti rigati
	* cooking spray
	* dried basil
	* flounder fillets
	* fresh parsley
	* garlic clove
	* onions
	* tomato sauce
	* tomatoes with juice
1. **Clam Dip**
| | |
| --- | --- |
| calories | 714.8 |
| carbohydrates | 38.2 |
| fat | 54.5 |
| protein | 0.0 |

	Ingredients:
	* clams
	* cream cheese
	* dry vegetable soup mix
	* frozen chopped spinach
	* mayonnaise
1. **How to Clean Clams**
| | |
| --- | --- |
| calories | 27.6 |
| carbohydrates | 5.9 |
| fat | 0.3 |
| protein | 0.0 |

	Ingredients:
	* clams
	* cold water
	* cornmeal
1. **New England Clam Chowder**
| | |
| --- | --- |
| calories | 418.2 |
| carbohydrates | 34.6 |
| fat | 20.1 |
| protein | 0.0 |

	Ingredients:
	* clams
	* evaporated milk
	* onions
	* pepper
	* potatoes
	* salt
	* salt pork
	* water
1. **Steamed Clams**
| | |
| --- | --- |
| calories | 445.9 |
| carbohydrates | 19.4 |
| fat | 4.4 |
| protein | 0.0 |

	Ingredients:
	* clams
	* kosher salt
	* red pepper flakes
	* shallot
	* white wine
1. **Legz\' Linguini W/ Red Clam Sauce**
| | |
| --- | --- |
| calories | 1805.8 |
| carbohydrates | 119.8 |
| fat | 100.1 |
| protein | 0.0 |

	Ingredients:
	* minced clams
	* parsley sprigs
	* real butter
	* salt
	* tomato paste
	* crushed red pepper flakes
	* dried basil
	* dried oregano
	* extra virgin olive oil
	* fresh parsley
	* garlic cloves
	* lemon juice
	* linguine
	* italian stewed tomatoes
	* clams
1. **Excellent Red Clam Sauce**
| | |
| --- | --- |
| calories | 187.3 |
| carbohydrates | 19.4 |
| fat | 7.9 |
| protein | 0.0 |

	Ingredients:
	* clams
	* crushed tomatoes
	* diced tomatoes in tomato puree
	* dried basil
	* dried oregano
	* dried thyme
	* garlic cloves
	* olive oil
	* onion
	* salt and pepper
1. **Stir-Fried Clams in Black Bean Sauce**
| | |
| --- | --- |
| calories | 169.5 |
| carbohydrates | 5.8 |
| fat | 11.0 |
| protein | 0.0 |

	Ingredients:
	* clams
	* cornstarch
	* sugar
	* thick soy sauce
	* water
	* fermented black beans
	* fresh gingerroot
	* garlic cloves
	* peanut oil
	* sesame oil
	* shaosing wine
	* spring onions
	* steamed rice
1. **Seafood Rice Casserole**
| | |
| --- | --- |
| calories | 328.4 |
| carbohydrates | 45.2 |
| fat | 8.6 |
| protein | 0.0 |

	Ingredients:
	* clams
	* cooked long   grain rice
	* evaporated milk
	* green chilies
	* onions
	* seasoned bread crumbs
	* shredded cheddar cheese
1. **Clams Oreganata**
| | |
| --- | --- |
| calories | 439.8 |
| carbohydrates | 21.8 |
| fat | 17.7 |
| protein | 0.0 |

	Ingredients:
	* clams
	* extra virgin olive oil
	* fresh lemon zest
	* fresh oregano
	* garlic cloves
	* plum tomatoes
	* toasted breadcrumbs
1. **Barbecued Clams With Corn Salsa (Bbq)**
| | |
| --- | --- |
| calories | 144.9 |
| carbohydrates | 21.7 |
| fat | 1.8 |
| protein | 0.0 |

	Ingredients:
	* clams in shell
	* fresh cilantro
	* whole corn
	* fresh green chile
	* garlic cloves
	* ground cumin
	* lime
	* lime wedges
	* salt
	* scallions
	* tomatoes
1. **Clam Salad With Miso Dresssing**
| | |
| --- | --- |
| calories | 53.9 |
| carbohydrates | 6.1 |
| fat | 0.6 |
| protein | 0.0 |

	Ingredients:
	* clams
	* green onions
	* light soy sauce
	* rice vinegar
	* sake
	* salt
	* sugar
	* water
	* white miso
1. **Clam-Vegetable Soup (Crock Pot)**
| | |
| --- | --- |
| calories | 126.2 |
| carbohydrates | 20.2 |
| fat | 1.3 |
| protein | 0.0 |

	Ingredients:
	* clams
	* dried marjoram
	* dried onion flakes
	* dried parsley flakes
	* frozen hash brown potatoes
	* frozen peas and carrots
	* salt
	* water
	* whole tomatoes
1. **Clam Linguine**
| | |
| --- | --- |
| calories | 720.2 |
| carbohydrates | 114.8 |
| fat | 14.1 |
| protein | 0.0 |

	Ingredients:
	* clams
	* condensed cream of mushroom soup
	* green onions
	* minced garlic cloves
	* olive oil
	* spaghetti
	* water
1. **Italian Flag Clams and Linguine Pasta Party**
| | |
| --- | --- |
| calories | 157.6 |
| carbohydrates | 8.1 |
| fat | 4.8 |
| protein | 0.0 |

	Ingredients:
	* cherrystone clams
	* clam juice
	* thyme
	* unsalted butter
	* clams
	* dry white wine
	* fresh pasta
	* garlic
	* pasta
	* red pepper flakes
	* salt and pepper
	* spinach pasta
1. **Clam Donburi**
| | |
| --- | --- |
| calories | 165.7 |
| carbohydrates | 7.1 |
| fat | 5.6 |
| protein | 0.0 |

	Ingredients:
	* clams
	* egg
	* ginger
	* mirin
	* parsley
	* sake
	* soy sauce
	* sugar
	* water
1. **Lori\'s Cataplana**
| | |
| --- | --- |
| calories | 752.4 |
| carbohydrates | 9.2 |
| fat | 52.8 |
| protein | 0.0 |

	Ingredients:
	* clams
	* crushed red pepper flakes
	* sea salt
	* dry white wine
	* garlic cloves
	* olive oil
	* onion
	* paprika
	* parsley
	* pork
1. **Cioppino Mediterranean**
| | |
| --- | --- |
| calories | 263.8 |
| carbohydrates | 14.3 |
| fat | 4.8 |
| protein | 0.0 |

	Ingredients:
	* clams
	* dried basil
	* shrimp
	* sole
	* tomato paste
	* water
	* whole tomatoes
	* dried oregano
	* dry red wine
	* garlic clove
	* green bell peppers
	* olive oil
	* onions
	* parsley
	* salt and pepper
1. **Crock Pot Clam Chowder**
| | |
| --- | --- |
| calories | 571.7 |
| carbohydrates | 38.3 |
| fat | 37.8 |
| protein | 0.0 |

	Ingredients:
	* clams
	* cornstarch
	* half   and   half
	* onion
	* pepper
	* potatoes
	* salt
	* salt pork
	* water
1. **Fettuccine With Bacon Clam Sauce**
| | |
| --- | --- |
| calories | 704.6 |
| carbohydrates | 46.2 |
| fat | 45.0 |
| protein | 0.0 |

	Ingredients:
	* bacon
	* butter
	* clams
	* dried oregano
	* fettuccine
	* fresh parsley
	* garlic cloves
	* heavy whipping cream
	* parmesan cheese
	* pepper
1. **Seared Clams With Brown Butter**
| | |
| --- | --- |
| calories | 309.3 |
| carbohydrates | 6.0 |
| fat | 24.2 |
| protein | 0.0 |

	Ingredients:
	* chives
	* fresh ground black pepper
	* lemon
	* olive oil
	* razor clams
	* salted butter
1. **Parellade (Paella Without Bones)**
| | |
| --- | --- |
| calories | 702.5 |
| carbohydrates | 70.6 |
| fat | 24.9 |
| protein | 0.0 |

	Ingredients:
	* broth
	* chicken livers
	* salt
	* shrimp
	* tomatoes
	* whole chicken breast
	* clams
	* frozen baby lima beans
	* garlic clove
	* lean pork
	* olive oil
	* pimiento
	* rice
	* saffron
1. **Slow Poke Jambalaya-Crock Pot**
| | |
| --- | --- |
| calories | 284.9 |
| carbohydrates | 14.6 |
| fat | 11.5 |
| protein | 0.0 |

	Ingredients:
	* beef broth
	* bell pepper
	* onion
	* oregano leaves
	* parsley
	* peas
	* salt
	* shrimp
	* smoked sausage
	* thyme leaves
	* cayenne
	* celery
	* chicken breasts
	* clam broth
	* clams
	* cooked rice
	* garlic clove
	* mussels
1. **Toasted Spaghetti With Clams**
| | |
| --- | --- |
| calories | 510.5 |
| carbohydrates | 68.1 |
| fat | 12.8 |
| protein | 0.0 |

	Ingredients:
	* bottled clam broth
	* crushed red pepper flakes
	* extra virgin olive oil
	* flat leaf parsley
	* garlic cloves
	* littleneck clams
	* pecorino romano cheese
	* spaghetti
	* water
1. **Dad\'s White Clam Sauce**
| | |
| --- | --- |
| calories | 882.8 |
| carbohydrates | 18.3 |
| fat | 58.6 |
| protein | 0.0 |

	Ingredients:
	* chicken broth
	* crushed red pepper flakes
	* extra virgin olive oil
	* garlic cloves
	* gluten   free pasta
	* minced clams
	* pepper
	* salt
1. **Wd Linguine With Clams & Parsley**
| | |
| --- | --- |
| calories | 439.9 |
| carbohydrates | 72.3 |
| fat | 2.6 |
| protein | 0.0 |

	Ingredients:
	* chicken broth
	* clams
	* salt
	* dry white wine
	* fresh ground pepper
	* fresh parsley
	* garlic
	* lemon
	* lemon zest
	* linguine
	* onion
1. **Shellfish Paella**
| | |
| --- | --- |
| calories | 599.0 |
| carbohydrates | 75.8 |
| fat | 15.2 |
| protein | 0.0 |

	Ingredients:
	* chicken broth
	* clams
	* plum tomatoes
	* red pepper flakes
	* roasted red peppers
	* saffron
	* salt and pepper
	* shrimp
	* unsalted butter
	* crab claws
	* dry white wine
	* frozen green peas
	* garlic cloves
	* green peppers
	* lobster tails
	* long grain rice
	* mussels
1. **Clam Sauce**
| | |
| --- | --- |
| calories | 171.5 |
| carbohydrates | 11.2 |
| fat | 5.0 |
| protein | 0.0 |

	Ingredients:
	* chicken broth
	* clams
	* salt
	* dry white wine
	* garlic
	* ground pepper
	* lemon
	* lemon zest
	* olive oil
	* onion
	* parsley
1. **One Pot Clambake**
| | |
| --- | --- |
| calories | 2703.8 |
| carbohydrates | 417.7 |
| fat | 62.0 |
| protein | 0.0 |

	Ingredients:
	* chicken broth
	* corn on the cob
	* sea salt
	* seafood
	* white wine
	* fresh ground pepper
	* fresh herbs
	* garlic cloves
	* olive oil
	* onion
	* red pepper flakes
	* red potatoes
	* sausage ring
1. **Tomato & Clam Soup**
| | |
| --- | --- |
| calories | 316.2 |
| carbohydrates | 32.9 |
| fat | 13.3 |
| protein | 0.0 |

	Ingredients:
	* chicken broth
	* clams
	* cooked pasta
	* dried basil
	* dried oregano
	* extra virgin olive oil
	* frozen spinach
	* garlic cloves
	* onion
	* tomatoes
1. **Clams Casino**
| | |
| --- | --- |
| calories | 109.3 |
| carbohydrates | 6.2 |
| fat | 3.9 |
| protein | 0.0 |

	Ingredients:
	* herb   ox very   low   sodium chicken broth
	* fat   free bread
	* fresh basil
	* fresh oregano
	* fresh parsley
	* garlic clove
	* hard   shelled clams
	* lemons
	* sodium   free margarine
	* turkey bacon



# hi

 ## my name is 

tony